In [1]:
import transformers
import datasets
import torch
import random
import copy
import itertools

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
model = transformers.AutoModelForPreTraining.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

/usr/local/lib64/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Example

In [3]:
texts = ["Koirat tykkäävät [MASK] kissoja."]

t = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
model_out = model(**t)
top20 = torch.argsort(model_out["prediction_logits"], dim=2, descending=True)[:,:,:20]
print("Guesses:",tokenizer.decode(top20[0,3]))   #index 3 as <s> is the first token (here we assumed all words==1 token)

Guesses: nuolla syödä haukkua katsella rakastaa tappaa katsoa kiusata varoa panna käsitellä myös purra leikkiä ruokkia olla sietää vihata hoitaa alistaa


## Global variables

In [4]:
special_tokens = tokenizer.all_special_tokens
continuation_marker = "##"   # how to get this?

## Masking

In [5]:

def mask_sentence(t):
    masked = []
    converted = tokenizer.convert_ids_to_tokens(t["input_ids"][0])
    need_to_mask = []
    for i in range(0, len(t["input_ids"][0])):
        masked.append(copy.deepcopy(t))
        if converted[i][:2] != continuation_marker:# and converted not in special_tokens:
            for k,j in itertools.product(need_to_mask, need_to_mask):
                masked[k]["input_ids"][0][j] = tokenizer.mask_token_id # mask the token in input
            need_to_mask=[i]
        else:
            need_to_mask.append(i)
    return masked

def mask_and_get_indices(t):
    masked = []
    converted = tokenizer.convert_ids_to_tokens(t["input_ids"][0])
    indices=[]
    for i in range(0, len(t["input_ids"][0])):
        if converted[i][:2] != continuation_marker:# and converted not in special_tokens:
            masked.append(copy.deepcopy(t))
            indices.append([i])
            masked[-1]["input_ids"][0][i] = tokenizer.mask_token_id # mask the token in input
        else:
            masked[-1]["input_ids"][0][i]= tokenizer.mask_token_id # mask the token in input
            indices[-1].append(i)
    return masked, indices

## Tokenizer

In [6]:
def wrap_tokenizer(tokenizer, options):
    """Wrapping to allow dataset.map() to have the tokenizer as a parameter"""
    def encode_dataset(d):
        """
        Tokenize the sentences.
        """
        output = tokenizer(d['text'], truncation= True, max_length=tokenizer.model_max_length)
        return output
    return encode_dataset


In [7]:
text = "Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995."

t = tokenizer(text, return_tensors='pt') # prepare normal tokenized input
print("tokens:", tokenizer.decode(t["input_ids"][0]))

masked = mask_sentence(t)

print("Masked:")
for m in masked:
    print(tokenizer.decode(m["input_ids"][0]))


tokens: [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
Masked:
[MASK] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] [MASK] [MASK], mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] [MASK] [MASK], mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa [MASK] mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa, [MASK] nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa, mun [MASK] on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa, mun nimi [MASK] Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa, mun nimi on [MASK] [MASK] ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[CLS] Heippa, mun nimi on [MASK] [MASK] ja harrastan pia

## Prediction

In [12]:
def to_probability(A):
    softmax = torch.nn.Softmax(dim=0)
    return softmax(A)

def predict(masked, i, top=5):
    model_out = model(**masked[i])
    logits = model_out["prediction_logits"]

    top_logits, top_tokens= torch.sort(logits, dim=2, descending=True)#[:,:,:top]
    top_probs = to_probability(top_logits[0,i,:])
    top_logits = top_logits[:,:,:top]
    top_tokens = top_tokens[:,:,:top]
    
    print("Guesses:",tokenizer.decode(top_tokens[0,i,:]))
    print("Logits: ",top_logits[0,i,:])
    print("Probs:  ",top_probs[:top])
    print(" ")
    return top_tokens, top_probs[:top]


for ind in range(len(masked)):
    print(tokenizer.decode(masked[ind]["input_ids"][0]))
    predict(masked,ind)

[MASK] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
Guesses: soitan ja. mulle soittaa
Logits:  tensor([10.7357, 10.7139, 10.2841,  9.8710,  9.8228], grad_fn=<SliceBackward0>)
Probs:   tensor([0.0640, 0.0626, 0.0408, 0.0270, 0.0257], grad_fn=<SliceBackward0>)
 
[CLS] [MASK] [MASK], mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
Guesses: Hei Moi Ain Niin Oon
Logits:  tensor([14.2579, 13.3853, 12.7012, 11.1238, 11.0604], grad_fn=<SliceBackward0>)
Probs:   tensor([0.3198, 0.1336, 0.0674, 0.0139, 0.0131], grad_fn=<SliceBackward0>)
 
[CLS] [MASK] [MASK], mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
Guesses: heiaoppakka
Logits:  tensor([10.6692, 10.5902, 10.1079, 10.0955, 10.0271], grad_fn=<SliceBackward0>)
Probs:   tensor([0.0558, 0.0516, 0.0319, 0.0315, 0.0294], grad_fn=<SliceBackward0>)
 
[CLS] Heippa [MASK] mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit

## In the article...

The probabilities are multiplied for multi-token words, with the beginning of the word unmasked.

In [9]:
# multiply the probablities 
import numpy as np



def to_probability(A):
    softmax = torch.nn.Softmax(dim=0)
    return softmax(A)

def predict(masked, top):
    model_out = model(**masked)
    logits = model_out["prediction_logits"]

    top_logits, top_tokens= torch.sort(logits, dim=2, descending=True)#[:,:,:top]
    top_probs = to_probability(top_logits[0,i,:])
    top_logits = top_logits[:,:,:top]
    top_tokens = top_tokens[:,:,:top]

    return top_tokens, top_probs[:top]


def get_scores(tokens,indices, top=5):
    for i in range(len(indices)):
        t = copy.deepcopy(tokens)
        current = indices[i:]
        for j in current:
            t["input_ids"][0][j] = tokenizer.mask_token_id
        predict(t,top)
        


def get_indices(t):
    converted = tokenizer.convert_ids_to_tokens(t["input_ids"][0])
    indices=[]
    for i in range(0, len(t["input_ids"][0])):
        if converted[i][:2] != continuation_marker:# and converted not in special_tokens:
            indices.append([i])
        else:
            indices[-1].append(i)
    return indices   


print("\nOther way:")
indices = get_indices(t)
for i in indices:
    print(i,tokenizer.decode(t["input_ids"][0]))

print("--------------------------------------------------")



    


Other way:
[0] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[1, 2] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[3] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[4] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[5] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[6] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[7, 8] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[9] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[10] [CLS] Heippa, mun nimi on Amanda ja harrastan pianon soittoa. Mulle voit soittaa 0442700995. [SEP]
[11, 12] [CLS] Heippa, mun nimi on Amanda ja harrastan

In [10]:
# note that this works differently for multiple dimensions

a = torch.tensor([[11.,12.,-5.,9.,2.],[22.,-32.,12.,5.,2.]])
b = to_probability(a)
print(b)

tensor([[1.6701e-05, 1.0000e+00, 4.1399e-08, 9.8201e-01, 5.0000e-01],
        [9.9998e-01, 7.7811e-20, 1.0000e+00, 1.7986e-02, 5.0000e-01]])
